# Feature engineering, rules-based models and ensembles 

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
import shap
from sklearn.model_selection import train_test_split
from wittgenstein import RIPPER
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.model_selection import GridSearchCV

### Import of clean data created in the first notebook

In [2]:
dataset = pd.read_csv('data/clean_data.csv')
dataset = dataset.drop(dataset.columns[0], axis=1)
test_dataset = pd.read_csv('data/test.csv').fillna(0)

In [3]:
def cross_validate_models(X, test_data, submission_file):
    labels = X['Target']
    cvm_data = X.drop(['Target', 'Date', 'Symbol', 'Id'], axis=1)
    models = {
        'GNB': GaussianNB(),
        'LR': LogisticRegression(max_iter=1000),
        'RFC': RandomForestClassifier(),
        'ETC': ExtraTreesClassifier(),
        'XGB': XGBClassifier()
    }

    folds = 5
    kf = KFold(n_splits=folds)
    best_model = None
    best_score = -1
    
    print("Algorithm\t\t", "\t".join([f"Fold {i+1}" for i in range(folds)]), "\tAverage")
    for model_name, model in models.items():
        scores = []
        for train_index, test_index in kf.split(cvm_data):
            X_train, X_test = cvm_data.iloc[train_index], cvm_data.iloc[test_index]
            y_train, y_test = labels.iloc[train_index], labels.iloc[test_index]
    
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            score = f1_score(y_test, y_pred)
            scores.append(score)
    
        avg_score = np.mean(scores)
        if avg_score > best_score:
            best_score = avg_score
            best_model = model

        print(f"{model_name}\t", "\t".join([f"{score:.2f}" for score in scores]), f"\t{avg_score:.2f}")

    X_test_set = test_data.drop(['Symbol', 'Date', 'Id'], axis=1).fillna(0)
    predictions = best_model.predict(X_test_set)
    
    submission_df = pd.DataFrame({
        'Id': test_data['Id'],
        'Predicted': predictions
    })

    submission_df.to_csv(f'{submission_file}.csv', index=False)
    
cross_val_data = dataset.copy()
test_data = test_dataset.copy()
cross_validate_models(cross_val_data, test_data, 'submission2')

Algorithm		 Fold 1	Fold 2	Fold 3	Fold 4	Fold 5 	Average
GNB	 0.88	0.90	0.84	0.89	0.87 	0.87
LR	 0.89	0.90	0.84	0.89	0.87 	0.88
RFC	 0.86	0.86	0.82	0.86	0.85 	0.85
ETC	 0.87	0.87	0.83	0.86	0.86 	0.86
XGB	 0.88	0.89	0.84	0.88	0.87 	0.87


## Feature creation

History related and domain specific features could prove to be useful as predictors so in the next chapter I will try to create new features that will enhance the accuracy of models the data is fed to.

### History related features

1. **Lagged Returns (Lagged_Return_1, Lagged_Return_5):**
   - These features capture the previous day's (Lagged_Return_1) and the return of the stock price five days ago (Lagged_Return_5).
   - `shift(1)` and `shift(5)` shift the 'Close' price downward by one and five rows, respectively, effectively capturing the past returns.

2. **Rolling Statistics (Rolling_Mean_5, Rolling_Std_5, Rolling_Min_5, Rolling_Max_5):**
   - These features calculate rolling statistics over a 5-day window for each stock symbol.
   - `x.rolling(window=5).mean()`, `x.rolling(window=5).std()`, `x.rolling(window=5).min()`, and `x.rolling(window=5).max()` compute the rolling mean, standard deviation, minimum, and maximum of the 'Close' price, respectively.

3. **Price Oscillations (Price_Oscillation):**
   - This feature computes the price oscillation, which is the ratio of the difference between the high and low prices to the closing price.
   - It provides insights into intraday price volatility and trading range.

4. **Volume Accumulation (Volume_Accumulation):**
   - This feature calculates the cumulative volume traded for each stock symbol over time.
   - `x.expanding().sum()` computes the expanding sum of the 'Volume' column, capturing the total volume accumulated until each point in time.

5. **Price to Moving Average Ratio (Price_to_SMA_20_Ratio):**
   - This feature calculates the ratio of the closing price to the 20-day Simple Moving Average (SMA).
   - It indicates whether the current price is above or below the average price trend over the specified period.

6. **Price Rate of Change (ROC):**
   - This feature computes the rate of change of the closing price over a 5-day period.
   - `x.pct_change(periods=5)` calculates the percentage change in the closing price relative to its value five days ago.

7. **Volatility Measures (Volatility_10):**
   - This feature computes the volatility of the stock price over a 10-day window.
   - It measures the standard deviation of daily percentage changes in the closing price.

8. **Price Crossings (Price_Above_SMA_50):**
   - This binary feature indicates whether the closing price is above the 50-day Simple Moving Average (SMA_50).
   - It can signal potential trend changes or breakout points.

9. **Relative Price Strength (Relative_Price_Strength):**
   - This feature calculates the ratio of the stock price to the average closing price of the market index on each date.
   - It measures the stock's relative performance compared to the broader market.

10. **Trading Volume Momentum (Volume_Momentum):**
   - This feature computes the momentum of trading volume over a 5-day period.
   - It measures the percentage change in trading volume relative to its value five days ago.

These features capture various aspects of historical price movements, volatility, and trading activity, providing valuable information for predictive modeling and trading strategies.

### Domain specific features

1. **Simple Moving Average (SMA_50):**
   - This line calculates the 50-day Simple Moving Average (SMA) for each stock symbol in the dataset. 
   - `data.groupby('Symbol')['Close']` groups the data by the 'Symbol' column and selects the 'Close' price for each group.
   - `.transform(lambda x: x.rolling(window=50).mean())` calculates the rolling mean (average) over a window of 50 periods for each group, which represents the 50-day SMA.

2. **Exponential Moving Average (EMA_20):**
   - This line calculates the 20-day Exponential Moving Average (EMA) for each stock symbol.
   - Similar to SMA, it calculates the rolling mean, but EMA gives more weight to recent prices.
   - `x.ewm(span=20, adjust=False).mean()` calculates the EMA with a span of 20 periods for each group.

3. **Relative Strength Index (RSI):**
   - RSI is a momentum oscillator that measures the speed and change of price movements.
   - The `calculate_rsi()` function calculates the RSI for each stock symbol using the closing prices and a specified window (default is 14 periods).
   - It calculates the average gain and loss over the specified window, computes the relative strength (RS), and then calculates the RSI using a formula.
   - `data.groupby('Symbol').apply(calculate_rsi)` applies this function to each group of data grouped by 'Symbol'.

4. **Bollinger Bands (SMA_20, std_20, upper_band, lower_band):**
   - Bollinger Bands are volatility bands placed above and below a moving average.
   - `SMA_20` calculates the 20-day SMA for each symbol.
   - `std_20` calculates the standard deviation of closing prices over a 20-day window for each symbol.
   - `upper_band` and `lower_band` are calculated by adding/subtracting two times the standard deviation from the SMA, providing a measure of volatility around the average.

5. **Volume Weighted Average Price (VWAP):**
   - VWAP is a trading benchmark used by traders that gives the average price a security has traded at throughout the day, based on both volume and price.
   - This line calculates VWAP by summing up the product of volume and average price (computed as the average of high, low, and close prices) over time, and dividing by the cumulative volume.

6. **Moving Average Convergence Divergence (MACD):**
   - MACD is a trend-following momentum indicator that shows the relationship between two moving averages of a security’s price.
   - `EMA_12` and `EMA_26` calculate the 12-day and 26-day EMA for each symbol, respectively.
   - `MACD` is computed as the difference between the 12-day and 26-day EMAs.

These features capture various aspects of price trends, volatility, and momentum, providing valuable insights for technical analysis in stock trading.

In [4]:
def calculate_rsi(close_prices, window=14):
    delta = close_prices.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window, min_periods=1).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window, min_periods=1).mean()

    rs = gain / loss
    rs[loss == 0] = 0

    rsi = 100 - (100 / (1 + rs))
    return rsi

def transform_data(data):
    data['SMA_50'] = data.groupby('Symbol')['Close'].transform(lambda x: x.rolling(50, min_periods=1).mean())
    
    data['Lagged_Return_1'] = data.groupby('Symbol')['Close'].shift(1)
    data['Lagged_Return_1'] = data['Lagged_Return_1'].bfill()
    
    data['Lagged_Return_5'] = data.groupby('Symbol')['Close'].shift(5)
    data['Lagged_Return_5'] = data['Lagged_Return_5'].bfill()
    
    data['Rolling_Mean_5'] = data.groupby('Symbol')['Close'].transform(lambda x: x.rolling(window=5, min_periods=1).mean())
    
    data['Rolling_Std_5'] = data.groupby('Symbol')['Close'].transform(lambda x: x.rolling(window=5, min_periods=1).std())
    data['Rolling_Std_5'] = data['Rolling_Std_5'].fillna(0)
    
    data['Rolling_Min_5'] = data.groupby('Symbol')['Close'].transform(lambda x: x.rolling(window=5, min_periods=1).min())
    
    data['Rolling_Max_5'] = data.groupby('Symbol')['Close'].transform(lambda x: x.rolling(window=5, min_periods=1).max())
    
    data['Price_Oscillation'] = (data['High'] - data['Low']) / data['Close']
    
    data['Volume_Accumulation'] = data.groupby('Symbol')['Volume'].transform(lambda x: x.expanding().sum())
    
    data['SMA_20'] = data.groupby('Symbol')['Close'].transform(lambda x: x.rolling(window=20, min_periods=1).mean())
    
    data['Price_to_SMA_20_Ratio'] = data['Close'] / data['SMA_20']
    
    data['ROC'] = data.groupby('Symbol')['Close'].pct_change(periods=5)
    data['ROC'] = data['ROC'].replace([np.inf, -np.inf], np.nan)
    data['ROC'] = data['ROC'].fillna(0)
    
    data['Volatility_10'] = data.groupby('Symbol')['Close'].pct_change().rolling(window=10, min_periods=1).std().fillna(0)
    
    data['Price_Above_SMA_50'] = (data['Close'] > data['SMA_50']).astype(int)
    
    market_index_data = data.groupby('Date')['Close'].mean().rename('Market_Index').reset_index()
    data = pd.merge(data, market_index_data, on='Date', how='left')
    data['Relative_Price_Strength'] = data['Close'] / data['Market_Index']
    
    data['Volume_Momentum'] = data.groupby('Symbol')['Volume'].pct_change(periods=5)
    data['Volume_Momentum'] = data['Volume_Momentum'].fillna(0)

    data['RSI'] = data.groupby('Symbol')['Close'].transform(lambda x: calculate_rsi(x))
    data['EMA_20'] = data.groupby('Symbol')['Close'].transform(lambda x: x.ewm(span=20, adjust=False).mean())
    data['std_20'] = data.groupby('Symbol')['Close'].transform(lambda x: x.rolling(window=20, min_periods=1).std())
    data['std_20'] = data['std_20'].fillna(0)
    data['upper_band'] = data['SMA_20'] + (2 * data['std_20'])
    data['lower_band'] = data['SMA_20'] - (2 * data['std_20'])
    data['VWAP'] = (data['Volume'] * (data['High'] + data['Low'] + data['Close']) / 3).cumsum() / data['Volume'].cumsum()
    data['EMA_12'] = data.groupby('Symbol')['Close'].transform(lambda x: x.ewm(span=12, adjust=False).mean())
    data['EMA_26'] = data.groupby('Symbol')['Close'].transform(lambda x: x.ewm(span=26, adjust=False).mean())
    data['MACD'] = data['EMA_12'] - data['EMA_26']
    
    return data

## Cross validating the models on the created feature set

To benchmark the quality of the newly created features multiple models will be cross-validated and their performance will then be compared.

In [ ]:
selected_features = ['Lagged_Return_1', 'Lagged_Return_5', 'Rolling_Mean_5', 'Rolling_Std_5', 'Rolling_Min_5', 'Rolling_Max_5', 'Price_Oscillation', 'SMA_20', 'EMA_20', 'Price_to_SMA_20_Ratio', 'ROC', 'Volatility_10', 'SMA_50', 'Price_Above_SMA_50', 'Relative_Price_Strength', 'Target', 'Date', 'Symbol', 'Id']

transformed_data = dataset.copy()
transformed_data = transform_data(transformed_data)[selected_features]
test_data = test_dataset.copy()
selected_features.remove('Target')
transformed_test_data = transform_data(test_data)[selected_features]

cross_validate_models(transformed_data.copy(), transformed_test_data.copy(), 'submission3')

Algorithm		 Fold 1	Fold 2	Fold 3	Fold 4	Fold 5 	Average
GNB	 0.88	0.90	0.84	0.89	0.86 	0.87


C:\Users\mpavlovic\Documents\fer\stock-trading-predictions\venv\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\mpavlovic\Documents\fer\stock-trading-predictions\venv\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://s

LR	 0.89	0.90	0.84	0.89	0.87 	0.88


## Wrapper feature selection

In [ ]:
def wrapper_feature_selection(data, features, target_features=12):
    y = data['Target']
    X = data.drop(['Target', 'Date', 'Symbol', 'Id'], axis=1)
    model = GaussianNB()
    remaining_features = features.copy()
    
    while len(remaining_features) > target_features:
        f1_scores = []
        for feature in remaining_features:
            X_copy = X.drop(feature, axis=1)
            model.fit(X_copy, y)
            y_pred = model.predict(X_copy)
            f1 = f1_score(y, y_pred)
            f1_scores.append(f1)

        best_idx = np.argmax(f1_scores)
        remaining_features.pop(best_idx)
        
    return remaining_features

In [ ]:
features = ['Lagged_Return_1', 'Lagged_Return_5', 'Rolling_Mean_5', 'Rolling_Std_5', 'Rolling_Min_5', 'Rolling_Max_5', 'Price_Oscillation', 'SMA_20', 'EMA_20', 'Price_to_SMA_20_Ratio', 'ROC', 'Volatility_10', 'SMA_50', 'Price_Above_SMA_50', 'Relative_Price_Strength']
best_features = wrapper_feature_selection(transformed_data, features)
best_features.extend(['Target', 'Date', 'Symbol', 'Id'])
selected_data = transformed_data[best_features]
best_features.remove('Target')
selected_test_data = transformed_test_data[best_features]
cross_validate_models(selected_data, selected_test_data, 'submission4')

In [ ]:
X = selected_data.drop(columns=['Target', 'Date', 'Symbol', 'Id'])
y = selected_data['Target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

best_ensemble_model = XGBClassifier()
best_ensemble_model.fit(X_train, y_train)

explainer = shap.TreeExplainer(best_ensemble_model)
shap_values = explainer.shap_values(X_train)

isolated_samples = selected_data.loc[selected_data.Id.isin(range(346055, 346065))]
isolated_samples.reset_index(drop=True, inplace=True)
for index, row in isolated_samples.iterrows():
    row_df = pd.DataFrame([row[X_train.columns]])
    sample_shap_values = explainer.shap_values(row_df)
    shap.force_plot(explainer.expected_value, sample_shap_values, row_df, matplotlib=True)

X = isolated_samples.drop(columns=['Target', 'Date', 'Symbol', 'Id'])
y = np.array(isolated_samples['Target'])

y_pred = best_ensemble_model.predict(X)
print(f'y:{y}\ny_predicted:{y_pred}')

## Ripper algorithm
Next part of the notebook performs the Ripper algorithm on AAPL stocks

### Grid search of prune size, dl_allowance and k hyperparameters

In [ ]:
feature_data_aapl = selected_data[selected_data['Symbol'] == 'AAPL']

X = feature_data_aapl.drop(columns=['Target', 'Date', 'Symbol', 'Id'])
y = feature_data_aapl['Target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

def ripper_grid_search(max_rules, max_cond):
    scorer = make_scorer(f1_score)
    param_grid = {
        'prune_size': [0.1, 0.2, 0.3],
        'dl_allowance': [0.1, 0.2, 0.3],
        'k': [2, 3, 4]
    }
    ripper = RIPPER(max_rules=max_rules, max_rule_conds=max_cond)
    grid_search = GridSearchCV(ripper, param_grid, scoring=scorer)
    grid_search.fit(X_train, y_train)
    best_params = grid_search.best_params_
    print("Best Parameters:", best_params)
    ripper_best = RIPPER(**best_params)
    ripper_best.fit(X_train, y_train)
    
    y_pred_best = ripper_best.predict(X_test)
    f1_best = f1_score(y_test, y_pred_best)
    acc_best = accuracy_score(y_test, y_pred_best)
    print("Best F1 Score:", f1_best)
    print("Ripper accuracy:", acc_best)
    print("Decision Rules:")
    for rule in ripper_best.ruleset_:
        print(rule)
        
ripper_grid_search(None, None)

### Maximum rule and conditions limitation

In [ ]:
ripper_grid_search(3, 2)